In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,GRU, LSTM
from keras import optimizers 
from sklearn.metrics import mean_squared_error

seed = 1234
np.random.seed(seed)
plt.style.use('ggplot')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = pd.read_csv("powerconsumption.csv", index_col="Datetime", parse_dates=["Datetime"])
factors = data[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows']]
target = data['PowerConsumption_Zone1']

In [3]:
# Normalize the data
scaler = MinMaxScaler()
scaled_factors = scaler.fit_transform(factors)
scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))

In [4]:
# Split the data into train, test, and validation sets
train_data, temp_data, train_target, temp_target = train_test_split(scaled_factors, scaled_target, test_size=0.2, random_state=42)
test_data, val_data, test_target, val_target = train_test_split(temp_data, temp_target, test_size=0.5, random_state=42)

In [5]:
# Reshape the data for the GRU model
train_data = np.reshape(train_data, (train_data.shape[0], 1, train_data.shape[1]))
test_data = np.reshape(test_data, (test_data.shape[0], 1, test_data.shape[1]))
val_data = np.reshape(val_data, (val_data.shape[0], 1, val_data.shape[1]))

In [7]:
# Build the GRU model
model = Sequential()
model.add(GRU(64, input_shape=(train_data.shape[1], train_data.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(train_data, train_target, epochs=50, batch_size=16, validation_data=(val_data, val_target))

Epoch 1/50
2621/2621 [==============================] - 9s 3ms/step - loss: 0.0321 - val_loss: 0.0269
Epoch 2/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0282 - val_loss: 0.0265
Epoch 3/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0275 - val_loss: 0.0267
Epoch 4/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0273 - val_loss: 0.0264
Epoch 5/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0271 - val_loss: 0.0261
Epoch 6/50
2621/2621 [==============================] - 8s 3ms/step - loss: 0.0270 - val_loss: 0.0265
Epoch 7/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0269 - val_loss: 0.0259
Epoch 8/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0268 - val_loss: 0.0258
Epoch 9/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.0267 - val_loss: 0.0260
Epoch 10/50
2621/2621 [==============================] - 7s 3ms/step - loss: 0.026

In [9]:
test_factor = pd.read_csv('output.csv')

,Temperature,Humidity,Windspeed,GeneralDiffuseFlows,DiffuseFlows
0,0.099687,2.897692,0.099687,2.897692,7.147650
1,0.099861,2.897238,0.099861,2.897238,7.148431
2,0.099695,2.898284,0.099695,2.898284,7.147984
3,0.099710,2.898194,0.099710,2.898194,7.148096
4,0.099835,2.898563,0.099835,2.898563,7.148403
5,0.099948,2.899177,0.099948,2.899177,7.147817


In [13]:
array_data = test_factor.to_numpy()

# Reshape the array to have a time step of 1
reshaped_array = array_data.reshape(-1, 1, 5)

In [28]:
# Predict the target values using the test set
predicted_target = model.predict(reshaped_array)


1/1 [==============================] - 0s 23ms/step


In [29]:
predicted_target

array([[[0.49042395]],

       [[0.4902262 ]],

       [[0.4905204 ]],

       [[0.49048874]],

       [[0.49051383]],

       [[0.49066514]]], dtype=float32)

In [35]:
array_2d = np.squeeze(predicted_target)
array = array_2d.reshape(-1, 1)
result = scaler.inverse_transform(array)

In [36]:
result

array([[32683.2  ],
       [32675.625],
       [32686.895],
       [32685.682],
       [32686.643],
       [32692.44 ]], dtype=float32)